In [1]:

import config
import dataset
import engine
import torch
import pandas as pd
import torch.nn as nn
import numpy as np
from tqdm import tqdm
from model import BERTBaseUncased
from sklearn import model_selection
from sklearn import metrics
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup


In [2]:

def run():
    
    dfx = pd.read_csv(config.TRAINING_FILE).fillna("none")
    dfx.sentiment = dfx.sentiment.apply(lambda x: 1 if x == "positive" else 0)

    df_train, df_valid = model_selection.train_test_split(
        dfx, test_size=0.1, random_state=42, stratify=dfx.sentiment.values
    )

    df_train = df_train.reset_index(drop=True)
    df_valid = df_valid.reset_index(drop=True)

    train_dataset = dataset.BERTDataset(
        review=df_train.review.values, target=df_train.sentiment.values
    )

    train_data_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=config.TRAIN_BATCH_SIZE, num_workers=4
    )

    valid_dataset = dataset.BERTDataset(
        review=df_valid.review.values, target=df_valid.sentiment.values
    )

    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=config.VALID_BATCH_SIZE, num_workers=1
    )

    device = torch.device("cuda")
    model = BERTBaseUncased()
    model.to(device)

    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {
            "params": [
                p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.001,
        },
        {
            "params": [
                p for n, p in param_optimizer if any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.0,
        },
    ]

    num_train_steps = int(len(df_train) / config.TRAIN_BATCH_SIZE * config.EPOCHS)
    optimizer = AdamW(optimizer_parameters, lr=3e-5)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=num_train_steps
    )

    best_accuracy = 0
    for epoch in range(config.EPOCHS):
        engine.train_fn(train_data_loader, model, optimizer, device, scheduler)
        outputs, targets = engine.eval_fn(valid_data_loader, model, device)
        outputs = np.array(outputs) >= 0.5
        accuracy = metrics.accuracy_score(targets, outputs)
        print(f"Accuracy Score = {accuracy}")
        if accuracy > best_accuracy:
            torch.save(model.state_dict(), config.MODEL_PATH)
            best_accuracy = accuracy


if __name__ == "__main__":
    run()


Some weights of the model checkpoint at C:/Users/User/OneDrive - Instituto Tecnológico Metropolitano/CODIGOS/Transformers/BERT-NER/Sentiment BERT/Input were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  1%|▊                                                       

KeyboardInterrupt: 

In [ ]:
print(torch.cuda.memory_allocated()/1024**2)

In [ ]:
#dfx=pd.read_csv(config.TRAINING_FILE).fillna("none")
#dfx.sentiment= dfx.sentiment.apply(
#lambda x: 1 if x=="positive" else 0)

In [ ]:
#dfx


In [ ]:
#dfx=pd.read_csv(config.TRAINING_FILE).fillna("none")

#dfx.sentiment= dfx.sentiment.apply(
  #  lambda x: 1 if x=="positive" else 0)
#df_train, df_valid= model_selection.train_test_split(
  #  dfx,
   # test_size=0.1,
   # random_state=42,
   # stratify= dfx.sentiment.values
   # )
#df_train= df_train.reset_index(drop=True)
#df_valid= df_valid.reset_index(drop=True)

#train_dataset=dataset.BERTDataset(
  #  review= df_train.review.values,
  #  target= df_train.sentiment.values
  #  )
    
#train_data_loader= torch.utils.data.DataLoader(
  #  train_dataset,
   # batch_size=config.TRAIN_BATCH_SIZE,
   # num_workers=4
   # )
    

In [ ]:
#for bi, d in tqdm(enumerate(train_data_loader),total=len(train_data_loader)):
   # break
    

In [ ]:
#d['ids'][0][0]*2